In [100]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from lxml import html
import urllib.request
from urllib.request import urlopen
import re
import wikipedia

# from pprint import pprint
# from lxml.html import fromstring
# import random
# import scrapy


In [101]:
        #quotes = [element.text for element in soup.find_all('a',{'class':'title'})]
        #list_of_quotes = [element.replace("\n", "") for element in quotes]
        #final_list.append(list_of_quotes)
        
        #authors = [element.text for element in soup.find_all('div',{'class':'author'})]
        #list_of_authors = [element.replace("\n", "") for element in authors]

In [99]:
def quote(url):
    list_of_quotes = []
    list_of_authors = []
    for i in range(1, 11):
        html = requests.get(url % i).content
        soup = BeautifulSoup(html)

        list_of_quotes.append([element.text.replace("\n", "") for element in soup.find_all('a',{'class':'title'})])
        merged_quotes = list(itertools.chain.from_iterable(list_of_quotes))
        
        list_of_authors.append([element.text.replace("\n", "") for element in soup.find_all('div',{'class':'author'})])
        merged_authors = list(itertools.chain.from_iterable(list_of_authors))
    
    #d = {'Quote':merged_quotes,'Author':merged_authors}
    df = pd.DataFrame({'Quote':merged_quotes,'Author':merged_authors})
    df.index = np.arange(1, len(df)+1)
    
    return df
    
quote('https://www.azquotes.com/top_quotes.html?p=%s')

,Quote,Author
1,The essence of strategy is choosing what not t...,Michael Porter
2,Death is something inevitable. When a man has ...,Nelson Mandela
3,"With or without religion, you would have good ...",Steven Weinberg
4,A person who never made a mistake never tried ...,Albert Einstein
5,"My mission in life is not merely to survive, b...",Maya Angelou
6,"In a world filled with hate, we must still dar...",Michael Jackson
7,Darkness cannot drive out darkness; only light...,"Martin Luther King, Jr."
8,To be a Christian means to forgive the inexcus...,C. S. Lewis
9,A man who is good enough to shed his blood for...,Theodore Roosevelt
10,Most folks are as happy as they make up their ...,Abraham Lincoln
